In [ ]:
import os
import json
import zipfile
import pickle
import numpy as np
from tempfile import TemporaryDirectory

def process_video_folder(video_folder_path, output_pickle_path):
    """
    Process a single video folder containing JSON files (one per frame).
    Extracts pose keypoints, left hand keypoints, and right hand keypoints and saves them into a pickle file.

    Parameters:
        video_folder_path (str): Path to the folder containing JSON files for one video.
        output_pickle_path (str): Path to save the pickle file for this video.
    """
    all_frames = {
        'pose': [],
        'hand_left': [],
        'hand_right': []
    }
    
    # List and sort all JSON files in the video folder
    json_files = [os.path.join(video_folder_path, f)
                  for f in os.listdir(video_folder_path) if f.endswith('.json')]
    json_files.sort()  # Ensure frames are in order

    for json_file in json_files:
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Skipping file {json_file} due to JSON decode error: {e}")
            continue

        # Check if the JSON file contains 'people' data
        if "people" not in data or not data["people"]:
            print(f"Skipping file {json_file} as no people data was found.")
            continue

        # Try to extract and reshape the pose and hand keypoints
        try:
            pose = np.array(data['people']['pose_keypoints_2d']).reshape(-1, 3)
            hand_left = np.array(data['people']['hand_left_keypoints_2d']).reshape(-1, 3)
            hand_right = np.array(data['people']['hand_right_keypoints_2d']).reshape(-1, 3)
        except KeyError as e:
            print(f"Skipping file {json_file} due to missing key: {e}")
            continue
        except Exception as e:
            print(f"Skipping file {json_file} due to error: {e}")
            continue

        all_frames['pose'].append(pose)
        all_frames['hand_left'].append(hand_left)
        all_frames['hand_right'].append(hand_right)

    # If no valid frames were found, skip saving a pickle file for this folder.
    if not all_frames['pose'] or not all_frames['hand_left'] or not all_frames['hand_right']:
        print(f"No valid frames found in folder {video_folder_path}. Skipping saving pickle.")
        return

    # Stack lists into NumPy arrays with shape (frames, keypoints, 3)
    all_frames['pose'] = np.stack(all_frames['pose'], axis=0)
    all_frames['hand_left'] = np.stack(all_frames['hand_left'], axis=0)
    all_frames['hand_right'] = np.stack(all_frames['hand_right'], axis=0)

    # Save the dictionary to a pickle file
    with open(output_pickle_path, 'wb') as f:
        pickle.dump(all_frames, f)

    print(f"Saved: {output_pickle_path}")

def process_all_videos_from_zip(zip_path, output_dir):
    """
    Process all video folders inside a ZIP archive.
    For each video folder, extract pose, left hand, and right hand keypoints from the JSON files 
    and save them to a separate pickle file.

    Parameters:
        zip_path (str): Path to the ZIP file containing the JSON data.
        output_dir (str): Directory where the output pickle files will be saved.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Extract the ZIP file into a temporary directory
    with TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Based on the structure: <temp_dir>/01_crowd_keypoint/01/ contains the video folders
        target_dir = os.path.join(temp_dir, "01") #, "01"
        if not os.path.exists(target_dir):
            print(f"Target directory {target_dir} does not exist. Check the ZIP file structure.")
            return

        # List all subdirectories (each representing one video)
        video_folders = [d for d in os.listdir(target_dir)
                         if os.path.isdir(os.path.join(target_dir, d))]
        for video_folder in video_folders:
            video_folder_path = os.path.join(target_dir, video_folder)
            # Save the pickle file with the original video folder name
            output_pickle_path = os.path.join(output_dir, f"{video_folder}.pkl")
            process_video_folder(video_folder_path, output_pickle_path)

# === Example usage ===

zip_file_path = r"/nas/Dataset/KSL/Train/[라벨]01_real_word_keypoint.zip"
output_directory = r"/nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/"

process_all_videos_from_zip(zip_file_path, output_directory)


Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD1417_REAL01_U.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD1770_REAL01_F.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD2620_REAL01_L.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD2174_REAL01_F.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD2981_REAL01_D.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD0398_REAL01_R.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD2583_REAL01_U.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD1057_REAL01_D.pkl
Saved: /nas/Chingiz/sing_language/all_json_to_pkl/01_real_word_keypoint/01/01/NIA_SL_WORD1810_REAL01_R.pkl
Saved: /nas/Chingiz/sing_language/all